In [2]:
%run 'functions_dump.py'
%run 'ks29_Get_folder_structure.ipynb'

In [3]:
structural_data = pd.read_hdf(files_dump_folder + 'structural_data_for_monomer.hdf', 'data')

In [8]:
colnames_His3_Scer_PLM = ['position1', 'amino_acid_1', 'position2', 'amino_acid_2', 'nothing', 'PML_score']
evolutionary_coupling = pd.read_table(files_dump_folder + 'evolutionary_coupling_analysis_with_evfold/ev_couplings/' + 'His3_Scer_PLM.txt',
             names=colnames_His3_Scer_PLM, sep=' ', header=None)

In [22]:
evolutionary_coupling['position_pair'] = evolutionary_coupling.apply(lambda df: frozenset(df[['position1', 'position2']].values), axis=1)
evolutionary_coupling[:4]

,position1,amino_acid_1,position2,amino_acid_2,nothing,PML_score,position_pair
0,28,L,29,A,0,1.158260,"(28, 29)"
1,28,L,30,I,0,0.810840,"(28, 30)"
2,28,L,31,E,0,0.317860,"(28, 31)"
3,28,L,32,H,0,0.313349,"(32, 28)"


In [ ]:
evolutionary_coupling.groupby('position_pair')['PML_score'].agg(np.max)

In [25]:
%run 'ks25_Read_epistasis.ipynb'


Reading precalculated epistasis...
/Users/karen/HIS3InterspeciesEpistasis/Analysis/Karen/files_dump/epistasis_in_foursomes/



Variables: 

- ep_dict: a dictionary, Keys are tuples (mutA, mutB) and values are pandas.Series of epistasis between mutA and mutB in all backgrounds. 
Tuples (mutA, mutB) are sorted by aa_position.

- get_ep_summary: function that takes ep_dict as an argument and applies an arbitrary function to every pd.Series object. For example, ep_summary = get_ep_summary(ep_dict, np.mean)

Important: natlib_filtering is currently True. This can be changed in the cell #2.



In [31]:
print structural_data.columns.values

['res_2' 'res_3' 'res_4' 'res_5' 'res_6' 'res_7' 'res_8' 'res_9' 'res_10'
 'res_11' 'res_12' 'res_13' 'res_14' 'res_15' 'res_16' 'res_17' 'res_18'
 'res_19' 'res_20' 'res_21' 'res_22' 'res_23' 'res_24' 'res_25' 'res_26'
 'res_27' 'res_28' 'res_29' 'res_30' 'res_31' 'res_32' 'res_33' 'res_34'
 'res_35' 'res_36' 'res_37' 'res_38' 'res_39' 'res_40' 'res_41' 'res_42'
 'res_43' 'res_44' 'res_45' 'res_46' 'res_47' 'res_48' 'res_49' 'res_50'
 'res_51' 'res_52' 'res_53' 'res_54' 'res_55' 'res_56' 'res_57' 'res_58'
 'res_59' 'res_60' 'res_61' 'res_62' 'res_63' 'res_64' 'res_65' 'res_66'
 'res_67' 'res_68' 'res_69' 'res_70' 'res_71' 'res_72' 'res_73' 'res_74'
 'res_75' 'res_76' 'res_77' 'res_78' 'res_79' 'res_80' 'res_81' 'res_82'
 'res_83' 'res_84' 'res_85' 'res_86' 'res_87' 'res_88' 'res_89' 'res_90'
 'res_91' 'res_92' 'res_93' 'res_94' 'res_95' 'res_96' 'res_97' 'res_98'
 'res_99' 'res_100' 'res_101' 'res_102' 'res_103' 'res_104' 'res_105'
 'res_106' 'res_107' 'res_108' 'res_109' 'res_110' 'r